In [163]:
# from google.colab import drive
# drive.mount('/content/drive')

In [164]:
#import library
import os
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import plotly.express as px
from palettable.colorbrewer.qualitative import Pastel1_7
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [165]:
#import data
netflix_titles = pd.read_csv('netflix_titles.csv')
netflix_titles.head(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [166]:
netflix_titles.tail(5)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."
8806,s8807,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...


In [167]:
print("Number of Rows: ", netflix_titles.shape[0], " data")
print("Number of Variables: ", netflix_titles.shape[1], " variable")

Number of Rows:  8807  data
Number of Variables:  12  variable


In [168]:
netflix_titles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [169]:
#categorical variable
#cat=['type', 'title', 'director', 'cast', 'country', 'date_added', 'rating', 'duration', 'listed_in']


cat_variable = netflix_titles.drop('show_id', axis=1)
cat_variable = netflix_titles.drop('release_year', axis=1)

cat_variable.describe()


,show_id,type,title,director,cast,country,date_added,rating,duration,listed_in,description
count,8807,8807,8807,6173,7982,7976,8797,8803,8804,8807,8807
unique,8807,2,8807,4528,7692,748,1767,17,220,514,8775
top,s1,Movie,Dick Johnson Is Dead,Rajiv Chilaka,David Attenborough,United States,"January 1, 2020",TV-MA,1 Season,"Dramas, International Movies","Paranormal activity at a lush, abandoned prope..."
freq,1,6131,1,19,19,2818,109,3207,1793,362,4


In [170]:
netflix_titles[netflix_titles['date_added']=='January 1, 2020']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
3051,s3052,Movie,Aruna & Her Palate,Edwin,"Dian Sastrowardoyo, Oka Antara, Hannah Al Rash...","Indonesia, South Korea, Singapore","January 1, 2020",2018,TV-14,106 min,"Dramas, International Movies",An epidemiologist turns her nationwide bird fl...
3052,s3053,TV Show,Because This Is My First Life,NaN,"Lee Min-ki, Jung So-min, Esom, Kim Ga-eun, Par...",South Korea,"January 1, 2020",2017,TV-14,1 Season,"International TV Shows, Korean TV Shows, Roman...",Two housemates get married for financial conve...
3053,s3054,TV Show,Border Security: America's Front Line,NaN,NaN,Canada,"January 1, 2020",2018,TV-14,1 Season,"Crime TV Shows, International TV Shows, Realit...",This reality series chronicles the behind-the-...
3054,s3055,Movie,Breaking the Bank,Vadim Jean,"Kelsey Grammer, Tamsin Greig, John Michael Hig...",United Kingdom,"January 1, 2020",2014,TV-PG,105 min,"Comedies, International Movies",Bank chairman Charles Bunbury – who knows next...
3055,s3056,TV Show,Cells at Work!,NaN,"Kana Hanazawa, Tomoaki Maeno, Daisuke Ono, Kik...",Japan,"January 1, 2020",2018,TV-PG,1 Season,"Anime Series, Teen TV Shows",One newcomer red blood cell just wants to do h...
...,...,...,...,...,...,...,...,...,...,...,...,...
8664,s8665,Movie,Up in the Air,Jason Reitman,"George Clooney, Vera Farmiga, Anna Kendrick, J...",United States,"January 1, 2020",2009,R,110 min,"Comedies, Dramas, Romantic Movies",Ryan Bingham flies around the country firing e...
8722,s8723,Movie,What Lies Beneath,Robert Zemeckis,"Harrison Ford, Michelle Pfeiffer, Diana Scarwi...",United States,"January 1, 2020",2000,PG-13,130 min,"Horror Movies, Thrillers",When Claire Spencer starts hearing ghostly voi...
8742,s8743,Movie,Wild Wild West,Barry Sonnenfeld,"Will Smith, Kevin Kline, Kenneth Branagh, Salm...",United States,"January 1, 2020",1999,PG-13,106 min,"Action & Adventure, Comedies, Sci-Fi & Fantasy","Armed with an ingenious arsenal, two top-notch..."
8745,s8746,Movie,Willy Wonka & the Chocolate Factory,Mel Stuart,"Gene Wilder, Jack Albertson, Peter Ostrum, Roy...","United States, East Germany, West Germany","January 1, 2020",1971,G,100 min,"Children & Family Movies, Classic Movies, Come...",Zany Willy Wonka causes a stir when he announc...


In [171]:
netflix_titles[netflix_titles['date_added']=='September 24, 2021']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...
6,s7,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",NaN,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...
7,s8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
8,s9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
9,s10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...
10,s11,TV Show,"Vendetta: Truth, Lies and The Mafia",NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, Docuseries, International TV S...","Sicily boasts a bold ""Anti-Mafia"" coalition. B..."


In [172]:
netflix_titles.isna().sum()

show_id            0
type               0
title              0
director        2634
cast             825
country          831
date_added        10
release_year       0
rating             4
duration           3
listed_in          0
description        0
dtype: int64

In [173]:
#drop variable director and cast
netflix_titles.drop(['director','cast'], axis=1, inplace=True)

#fix missing value in the variable country
netflix_titles['country'] = netflix_titles['country'].fillna(netflix_titles['country'].mode()[0])
print('Now we have ',netflix_titles.shape[1], 'varibles')

Now we have  10 varibles


In [174]:
# Lets retrieve just the first country
netflix_titles['principal_country'] = netflix_titles['country'].apply(lambda x: x.split(",")[0])
print('after we are include variable principal_country, we have ',netflix_titles.shape[1], 'variables')

after we are include variable principal_country, we have  11 variables


In [175]:
netflix_titles[netflix_titles['duration'].isna()]

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description,principal_country
5541,s5542,Movie,Louis C.K. 2017,United States,"April 4, 2017",2017,74 min,NaN,Movies,"Louis C.K. muses on religion, eternal love, gi...",United States
5794,s5795,Movie,Louis C.K.: Hilarious,United States,"September 16, 2016",2010,84 min,NaN,Movies,Emmy-winning comedy writer Louis C.K. brings h...,United States
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,United States,"August 15, 2016",2015,66 min,NaN,Movies,The comic puts his trademark hilarious/thought...,United States


In [176]:
#replace NaN value of duration variable with rating
netflix_titles['duration'][netflix_titles[netflix_titles['duration'].isna()].index] = netflix_titles['rating'][netflix_titles[netflix_titles['duration'].isna()].index]
#remove data entry errors in variable rating
netflix_titles['rating'] = netflix_titles['rating'].replace(['74 min', '84 min', '66 min'], np.NaN)
netflix_titles['duration'].isna().sum()

C:\Users\khiem\AppData\Local\Temp\ipykernel_18276\7747708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  netflix_titles['duration'][netflix_titles[netflix_titles['duration'].isna()].index] = netflix_titles['rating'][netflix_titles[netflix_titles['duration'].isna()].index]


0

In [177]:
netflix_titles['rating'].unique()

array(['PG-13', 'TV-MA', 'PG', 'TV-14', 'TV-PG', 'TV-Y', 'TV-Y7', 'R',
       'TV-G', 'G', 'NC-17', nan, 'NR', 'TV-Y7-FV', 'UR'], dtype=object)

In [178]:
netflix_titles[netflix_titles['rating'].isna()]

,show_id,type,title,country,date_added,release_year,rating,duration,listed_in,description,principal_country
5541,s5542,Movie,Louis C.K. 2017,United States,"April 4, 2017",2017,NaN,74 min,Movies,"Louis C.K. muses on religion, eternal love, gi...",United States
5794,s5795,Movie,Louis C.K.: Hilarious,United States,"September 16, 2016",2010,NaN,84 min,Movies,Emmy-winning comedy writer Louis C.K. brings h...,United States
5813,s5814,Movie,Louis C.K.: Live at the Comedy Store,United States,"August 15, 2016",2015,NaN,66 min,Movies,The comic puts his trademark hilarious/thought...,United States
5989,s5990,Movie,13TH: A Conversation with Oprah Winfrey & Ava ...,United States,"January 26, 2017",2017,NaN,37 min,Movies,Oprah Winfrey sits down with director Ava DuVe...,United States
6827,s6828,TV Show,Gargantia on the Verdurous Planet,Japan,"December 1, 2016",2013,NaN,1 Season,"Anime Series, International TV Shows","After falling through a wormhole, a space-dwel...",Japan
7312,s7313,TV Show,Little Lunch,Australia,"February 1, 2018",2015,NaN,1 Season,"Kids' TV, TV Comedies","Adopting a child's perspective, this show take...",Australia
7537,s7538,Movie,My Honor Was Loyalty,Italy,"March 1, 2017",2015,NaN,115 min,Dramas,"Amid the chaos and horror of World War II, a c...",Italy


In [179]:
#get index of 'rating'
netflix_titles.columns.get_loc('rating')

6

In [180]:
#fix missing value rating
rating_replacements = {
    5541: 'TV-MA',
    5794: 'NR',
    5813: 'TV-MA',
    5989: 'TV-PG',
    6827: 'TV-14',
    7312: 'TV-MA',
    7537: 'TV-MA '
}

for id, rate in rating_replacements.items():
    netflix_titles.iloc[id, 6] = rate
    
netflix_titles['rating'].isna().sum()

0

In [181]:
#fix missing value in the variable date_added
netflix_titles=netflix_titles.dropna(axis=0)
# axis=0 for drop rows and axis=1 for drop column
print('Now we have ', netflix_titles.shape[0], ' data for the analysis')
print('And we have ', netflix_titles.shape[1], ' variable for the analysis')

Now we have  8797  data for the analysis
And we have  11  variable for the analysis


In [182]:
netflix_show = netflix_titles[netflix_titles['type']=='TV Show']
netflix_movies = netflix_titles[netflix_titles['type']=='Movie']

### **Cosine with Description**

In [183]:
vectorizer = TfidfVectorizer()
transform = vectorizer.fit_transform(netflix_titles['description'])
cs = cosine_similarity(transform, transform)

In [184]:
data = netflix_titles['title']

def netflix_recommendations_description(title, cs=cs):
  title_search = pd.Index(data).get_loc(title)
  sim = list(enumerate(cs[title_search]))
  sim = sorted(sim, key=lambda x: x[1], reverse = True)
  top_ten = sim[1:11]
  recom = [i[0] for i in top_ten]
  return netflix_titles[['title', 'description']].iloc[recom]

### **Running Model (cosine with description)**

In [185]:
netflix_recommendations_description('Sankofa')

,title,description
8353,The Hunt,Witness some of the most riveting showdowns in...
7037,I Am Jane Doe,This intense documentary follows real cases of...
3619,I'm Brent Morin,"In a witty solo show, Brent Morin serves up in..."
5044,When We First Met,Using a magical photo booth that sends him bac...
5580,One More Time,This fantasy drama follows an indie band singe...
7700,Palm Trees in the Snow,"Finding a tantalizing clue in an old letter, a..."
3094,The App,"While in Rome to shoot his first movie, actor ..."
2824,The Valhalla Murders,An Oslo detective with a painful past returns ...
5653,My Runway,An egotistic top male model and a pretty model...
7904,Running for Grace,"In 1920s Hawaii, a young man of mixed race and..."


### **Cosine with genre**

In [186]:
vectorizer_genre = TfidfVectorizer()
transform_genre = vectorizer_genre.fit_transform(netflix_titles['listed_in'])
cs_genre = cosine_similarity(transform_genre, transform_genre)

In [187]:
data = netflix_titles['title']

def netflix_recommendations_genre(title, cs=cs):
  title_search_genre = pd.Index(data).get_loc(title)
  sim_genre = list(enumerate(cs_genre[title_search_genre]))
  sim_genre = sorted(sim_genre, key=lambda x: x[1], reverse = True)
  top_ten_genre = sim_genre[1:11]
  recom_genre = [i[0] for i in top_ten_genre]
  return netflix_titles[['title',  'listed_in']].iloc[recom_genre]

### **Running Model (cosine with genre)**

In [188]:
netflix_recommendations_genre('Sankofa')

,title,listed_in
30,Ankahi Kahaniya,"Dramas, Independent Movies, International Movies"
206,"LSD: Love, Sex Aur Dhokha","Dramas, Independent Movies, International Movies"
282,La diosa del asfalto,"Dramas, Independent Movies, International Movies"
339,I missed you: Director's Cut,"Dramas, Independent Movies, International Movies"
365,Eyes of a Thief,"Dramas, Independent Movies, International Movies"
371,Pomegranates and Myrrh,"Dramas, Independent Movies, International Movies"
394,African America,"Dramas, Independent Movies, International Movies"
633,Into the Wind,"Dramas, Independent Movies, International Movies"
698,Fan Girl,"Dramas, Independent Movies, International Movies"
750,Tragic Jungle,"Dramas, Independent Movies, International Movies"


In [189]:
netflix_recommendations_genre('Zodiac')

,title,listed_in
7667,Only God Forgives,"Cult Movies, Independent Movies, Thrillers"
6753,Final Destination,"Cult Movies, Horror Movies, Thrillers"
1862,Disco Dancer,"Cult Movies, Dramas, International Movies"
6147,American Psycho,"Comedies, Cult Movies, Dramas"
6450,Chasing Amy,"Comedies, Cult Movies, Dramas"
6743,Fear and Loathing in Las Vegas,"Comedies, Cult Movies, Dramas"
7944,Satte Pe Satta,"Comedies, Cult Movies, Dramas"
8072,Spring Breakers,"Comedies, Cult Movies, Dramas"
8614,Trainspotting,"Comedies, Cult Movies, Dramas"
144,House Party,"Comedies, Cult Movies"


In [190]:
netflix_recommendations_genre('Stranger Things')

,title,listed_in
3187,Nightflyers,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy"
3685,Stranger Things,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy"
6953,Helix,"TV Horror, TV Mysteries, TV Sci-Fi & Fantasy"
241,Manifest,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy"
3986,The OA,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy"
5287,The Vampire Diaries,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy"
5939,The 4400,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy"
8421,The Messengers,"TV Dramas, TV Mysteries, TV Sci-Fi & Fantasy"
2190,The Umbrella Academy,"TV Action & Adventure, TV Mysteries, TV Sci-Fi..."
2303,Warrior Nun,"TV Action & Adventure, TV Mysteries, TV Sci-Fi..."


In [192]:
joblib.dump(netflix_recommendations_genre, './genre.pkl')
joblib.dump(netflix_recommendations_description, './description.pkl')

['./description.pkl']